File for finding the metrics of each of the graphs at different distances.

In [ ]:
import os
import re
import pandas as pd
import igraph as ig
import numpy as np
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point, LineString
import geopandas as gpd
import graph_functions as graf

In [ ]:
def download_classified_thumbnails(folder_directory, data_folder, shp_dir, 
                                   n_clusters = 10, fishnet_rows = 5, 
                                   fishnet_cols = 5, scale = 1, xmeans = False, error_log = False):
    
    # cleaning out any extra files in case ran previously
    delete_file_path = f'{shp_dir}/'
    delete_file_paths = os.listdir(delete_file_path)
    for file in delete_file_paths:
        if (re.search(r'[0-9]_gcs+', file)) or (re.search(r'[a-zA-Z]_gcs+', file)):
            full_path = f'{delete_file_path}{file}'
            print(full_path)
            os.remove(full_path)
            print(f'removed {file}')
    
    # gaining a list of all the files in the given folder
    shp_files = [f for f in os.listdir(shp_dir) if f.endswith('.shp')]

    count = 0
    # Iterate directory
    for path in os.listdir(shp_dir):
        # check if current path is a file
        if path.endswith('.shp'):
            count += 1
    print(f'Downloading data for {count} files')
    
    index_num = -1
    for shp_file in shp_files:
        # increase the index by one for file searching and for creation later
        index_num = index_num + 1
        file_name_split = shp_file.split(".")
        
        # create the path to the start shapefile (polygon area)
        shp_path = os.path.join(shp_dir, shp_file)
        # create a feature collection from the feature
        feature_collection = geemap.shp_to_ee(shp_path)

In [ ]:
def global_metrics_df(given_directory):
    # graph_type
    threshold_dist = []
    # to easily find the year
    years = []
    densitys = []
    avg_path_lengths = []
    diameters = []
    transitivitys = []
    clusters = []
    mean_clusters = []
    max_cluster_sizes = []

    adjacency_files = [f for f in os.listdir(given_directory) if f.endswith('.adjacency')]

    for adj_file in adjacency_files:
        # use regex terms to find the year and the threshold distance
